#Import Libraries & Packages

In [469]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import SCORERS

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

pd.pandas.set_option('display.max_columns',None)


# Import Processed Data
#### filename='data_processed2.csv

In [470]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [471]:
final_df=pd.read_csv("/content/drive/MyDrive/NFT Price Prediction/dataset/data_processed2.csv")
test=pd.read_csv('/content/drive/MyDrive/NFT Price Prediction/dataset/nfts_predict.csv')

In [472]:
final_df.head(5)

,Unnamed: 0,global_index,nft_id,collection_id,rarity_score,openrarity_score,last_sale_date,last_sale_price,n_tweets_in_range,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets,total_supply,creation_date,verification_status,n_of_traits,contract_type,seller_fees,openrarity_enabled,has_discord,has_medium,sale_year,sale_month,creation_to_last_sale,openrarity_rankRare_var,15000.0,15555.0,5555.0,8849.0,9928.0,9999.0,nan,openrarity_max_rankRare_var
0,0,-2.432025,-6.999358,0.813622,-2.940693,-0.561648,0.213869,1.266732,0.947537,1.439493,0.380163,1.201866,-0.747717,-0.640385,0.082130,1.398811,0.446277,1.400544,-12.259114,0.867180,-1.089063,-2.375507,0.880729,1.037420,-0.565829,0.820615,-0.326514,0.342080,0.958217,0.422459,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
1,1,-2.026188,-6.999358,0.728050,-0.468541,-0.561648,-2.127903,1.200357,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.394779,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,-2.923295,0.363493,-0.567870,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
2,2,-2.026157,-6.428333,0.728050,-0.213949,-0.561648,0.923315,3.180572,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.394779,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,0.342080,0.363493,0.741303,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
3,3,-2.026125,-6.094305,0.728050,-0.275608,-0.561648,-2.127903,3.502910,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.394779,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,-2.923295,0.363493,-0.567870,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
4,4,-2.026094,-5.857308,0.728050,-0.366154,-0.561648,0.757695,4.009567,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.394779,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,0.342080,0.581304,0.780392,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551


In [473]:
final_df.drop(columns=['Unnamed: 0'],inplace=True)

In [474]:
df_Train=final_df.iloc[:final_df.shape[0]-test.shape[0],:]
df_Test=final_df.iloc[final_df.shape[0]-test.shape[0]:,:]

In [475]:
(df_Train.shape,df_Test.shape)

((309910, 38), (181214, 38))

In [476]:
df_Test.drop(['last_sale_price'],axis=1,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [477]:
X_train=df_Train.drop(['last_sale_price'],axis=1)
y_train=df_Train['last_sale_price']

X_full=X_train.copy()
y_full=y_train.copy()
    
X_train,x_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size = 0.1,random_state=1)

# Modeling

In [478]:
reg_ols= sm.OLS(np.log(y_train), X_train)
results = reg_ols.fit()
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
=============================================================================
Model:                  OLS                Adj. R-squared:       0.541       
Dependent Variable:     last_sale_price    AIC:                  1175013.4801
Date:                   2023-02-27 07:39   BIC:                  1175382.3338
No. Observations:       278919             Log-Likelihood:       -5.8747e+05 
Df Model:               34                 F-statistic:          9674.       
Df Residuals:           278884             Prob (F-statistic):   0.00        
R-squared:              0.541              Scale:                3.9540      
-----------------------------------------------------------------------------
                             Coef.  Std.Err.     t     P>|t|   [0.025  0.975]
-----------------------------------------------------------------------------
global_index                -0.2711   0.0143  -18.9828 0.0000 -0.2991 -0.2431
nft_id                      -0.0339   0.0046   -7.3193 0.0000 -0.0429 -0.0248
collection_id               -0.2420   0.0146  -16.5586 0.0000 -0.2706 -0.2134
rarity_score                -0.0488   0.0126   -3.8663 0.0001 -0.0736 -0.0241
openrarity_score             0.0683   0.0331    2.0631 0.0391  0.0034  0.1332
last_sale_date               0.2768   0.0089   31.2572 0.0000  0.2594  0.2942
n_tweets_in_range            0.3354   0.0183   18.3136 0.0000  0.2995  0.3713
avg_likes                    0.4222   0.0846    4.9924 0.0000  0.2564  0.5879
avg_replies                 -3.7504   0.0642  -58.4443 0.0000 -3.8762 -3.6246
avg_retweets                 1.7665   0.0801   22.0438 0.0000  1.6094  1.9235
min_likes                   -1.7703   0.0249  -71.1754 0.0000 -1.8190 -1.7215
min_replies                  3.4948   0.0357   97.8711 0.0000  3.4248  3.5648
min_retweets                 0.0414   0.0285    1.4532 0.1462 -0.0144  0.0972
max_likes                    1.8411   0.0863   21.3253 0.0000  1.6719  2.0103
max_replies                  0.7411   0.0471   15.7391 0.0000  0.6488  0.8334
max_retweets                -0.4045   0.0757   -5.3430 0.0000 -0.5529 -0.2561
total_supply                 1.0606   0.0183   57.8363 0.0000  1.0247  1.0966
creation_date               -0.3677   0.0088  -41.6541 0.0000 -0.3850 -0.3504
verification_status          0.4697   0.0269   17.4567 0.0000  0.4170  0.5225
n_of_traits                 -0.5936   0.0158  -37.6871 0.0000 -0.6245 -0.5628
contract_type                1.3039   0.0227   57.3414 0.0000  1.2594  1.3485
seller_fees                 -1.1826   0.0111 -106.1737 0.0000 -1.2045 -1.1608
openrarity_enabled          -0.7583   0.0167  -45.4626 0.0000 -0.7910 -0.7256
has_discord                  0.4781   0.0237   20.1296 0.0000  0.4315  0.5246
has_medium                  -0.1185   0.0161   -7.3597 0.0000 -0.1501 -0.0869
sale_year                   -0.0423   0.0095   -4.4787 0.0000 -0.0609 -0.0238
sale_month                  -0.1769   0.0067  -26.2827 0.0000 -0.1901 -0.1637
creation_to_last_sale       -0.2353   0.0066  -35.5689 0.0000 -0.2482 -0.2223
openrarity_rankRare_var     -0.7583   0.0167  -45.4626 0.0000 -0.7910 -0.7256
15000.0                      0.0749   0.0188    3.9882 0.0001  0.0381  0.1117
15555.0                      0.9987   0.0194   51.5839 0.0000  0.9608  1.0367
5555.0                       0.8461   0.0153   55.4170 0.0000  0.8162  0.8761
8849.0                       0.6166   0.0139   44.2329 0.0000  0.5893  0.6439
9928.0                       0.6742   0.0192   35.0857 0.0000  0.6365  0.7119
9999.0                      -0.2041   0.0121  -16.9110 0.0000 -0.2277 -0.1804
nan                          0.7583   0.0167   45.4626 0.0000  0.7256  0.7910
openrarity_max_rankRare_var 18.0544   0.0589  306.3247 0.0000 17.9388 18.1699
-----------------------------------------------------------------------------
Omnibus:              40190.861      Durbin-Watson:         1.999         

In [479]:
y_v_sm=np.exp(results.predict(x_valid))
y_t_sm=np.exp(results.predict(df_Test))

In [480]:
def rmse(predictions, targets):
    r= np.sqrt(((predictions - targets) ** 2).mean())
    return np.exp(-r/10 )

In [481]:
rmse(y_v_sm,y_valid)

0.3013259900441942

## Extra Trees regressor

In [482]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor(max_depth=4, min_samples_leaf=10, min_samples_split=25,
                    n_jobs=4)

# etr.fit(X_train,np.log(y_train))

In [483]:
# y_v_etr=np.exp(etr.predict(x_valid))
# y_t_etr=np.exp(etr.predict(df_Test))
# y_t_etr

In [484]:
# y_v_etr

In [485]:
# rmse(y_v_etr,y_valid)

### Hyper parameter Tuning

In [486]:
n_estimators = [40,55]
max_depth = [12,4,5,8,2]
min_samples_leaf=[10,20,40,80]
# min_samples_split=[5,15,25,35,45,60]


# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'min_samples_leaf':min_samples_leaf
    }

In [487]:
from sklearn.model_selection import RandomizedSearchCV 
random_cv_etr = RandomizedSearchCV(estimator=etr,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=15,
            scoring = 'neg_root_mean_squared_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [488]:
# random_cv_etr.fit(np.array(X_train[:30000]),np.log(y_train[:30000]))

In [489]:
# etr = ExtraTreesRegressor(min_samples_split=25,n_jobs=4)


## GBM

In [490]:
gbrt=gbrt = GradientBoostingRegressor(max_depth=5, min_samples_split=200, n_estimators=50,
                          random_state=42)
# gbrt.fit(X_train, np.log(y_train))

In [491]:
# y_t_gbr=np.exp(gbrt.predict(df_Test))
    
# y_v_gbr=np.exp(gbrt.predict(x_valid))
# rmse(y_v_gbr,y_valid)

In [492]:
# y_t_gbr

### Hyper parameter Tuning

In [493]:
n_estimators = [40, 45,50, 55,60,80]
max_depth = [ 4, 5, 6]
learning_rate=[0.1,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23]
min_samples_split=[100,125,150,200,250]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    # 'learning_rate':learning_rate,
    'min_samples_split':min_samples_split
    }

In [494]:
from sklearn.model_selection import RandomizedSearchCV 
random_cv_gbm = RandomizedSearchCV(estimator=gbrt,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=15,
            scoring = 'neg_root_mean_squared_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [495]:
# random_cv_gbm.fit(X_train[:30000],np.log(y_train[:30000]))

In [496]:
# random_cv_gbm.best_estimator_

## Light GBM

In [497]:
light_gbm = LGBMRegressor(bagging_fraction=0.9, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.75, feature_fraction_seed=7, lambda_l1=2,
              learning_rate=0.2, max_bin=200, max_depth=4, min_data_in_leaf=10,
              n_estimators=80, num_leaves=40, objective='regression',
              verbose=-1)

In [498]:
# light_gbm.fit(X_train,np.log(y_train))

In [499]:
# y_v_lgbm=np.exp(light_gbm.predict(x_valid))
# y_t_lgbm=np.exp(light_gbm.predict(df_Test))

In [500]:
# rmse(y_valid,y_v_lgbm)

In [501]:
# y_t_lgbm

### Hyper parameter Tuning

In [502]:
num_leaves = [2,3,4,5, 10, 20, 30,40, 50, 100, 150]
max_depth = [2, 4, 5, 6,8,10,12,14]
learning_rate=[0.01,0.02,0.03,0.05,0.1,0.20,]
min_data_in_leaf=[10,20,30,40, 50]
bagging_fraction= [0.7, 0.8, 0.9]
feature_fraction=[0.2, 0.5, 0.75]
lambda_l1=[0, 1, 2,10]
n_estimators = [30, 40,50, 60,70,80,100,200]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_data_in_leaf':min_data_in_leaf,
    'bagging_fraction':bagging_fraction,
    'feature_fraction':feature_fraction,
    'lambda_l1':lambda_l1,
    'num_leaves':num_leaves

    }

In [503]:
from sklearn.model_selection import RandomizedSearchCV 
random_cv_lgbm = RandomizedSearchCV(estimator=light_gbm,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=30,
            scoring = 'neg_root_mean_squared_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [504]:
# random_cv_lgbm.fit(X_train,np.log(y_train))

In [505]:
# random_cv_lgbm.best_estimator_

## XGBoost

In [506]:
regressor=XGBRegressor(gpu_id=1, interaction_constraints='', max_depth=5,
             min_child_weight=3, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3)

In [507]:
# y_t_xgb=np.exp(regressor.predict(df_Test))
    
# y_v_xgb=np.exp(regressor.predict(x_valid))
# rmse(y_v_xgb,y_valid)

In [508]:
# y_t_xgb

### Hyper parameter Tuning

In [509]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [510]:
## Hyper Parameter Optimization


n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [511]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [512]:
random_cv_xgb = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=20,
            scoring = 'neg_root_mean_squared_error',n_jobs = 5,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [513]:
# random_cv_xgb.fit(X_train[:20000],y_train[:20000])

In [514]:
# random_cv_xgb.best_estimator_

## Generic Panel Regression

### Single Algorithm Based Panel Regression


In [515]:
def panel(df,alg,variable,error_func,pred_features,test):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    # df_temp=df_temp.drop(columns=[variable])
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    # print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]

      alg.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=alg.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')


  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  alg.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=alg.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


### Tuned Panel Regression


In [516]:
def panel_tune(df,alg,variable,error_func,pred_features,test,hyp):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    # print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]
      hyp.fit(x_train_temp,y_train_temp)
      best_est=hyp.best_estimator_
      print(best_est)
      best_est.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=best_est.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')


  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  hyp.fit(x_train_temp,y_train_temp)
  best_est=hyp.best_estimator_
  print(best_est)
  best_est.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=best_est.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


### Pre tuned Panel Regression


In [517]:
def panel_pre_tune(df,tuned,variable,error_func,pred_features,test):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    # df_temp=df_temp.drop(columns=[variable])
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]
      alg=tuned[i]
      print(alg)
      alg.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=alg.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  print(df_train_temp.shape,df_test_temp.shape)
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  # hyp.fit(x_train_temp,y_train_temp)
  # best_est=hyp.best_estimator_
  # print(best_est)
  alg=tuned[24]
  print(alg)
  alg.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=alg.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


### Clustering to create panels

In [518]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

km = KMeans(n_clusters=25, random_state=1)
new =final_df.drop(columns=['last_sale_price'])

In [519]:
# visualizer = KElbowVisualizer(km, k=(15,30))
# visualizer.fit(new)        # Fit the data to the visualizer
# visualizer.show()

In [520]:
km.fit(np.array(new))
predict=km.predict(new)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [521]:
final_df['cluster']=predict

In [522]:
final_df['cluster'].value_counts()

3     48553
20    48256
1     40777
22    34904
23    29899
15    26925
12    24188
19    20216
4     20024
9     19602
8     18627
16    16205
11    15555
7     15000
13    14009
17    12451
0     11110
21    10000
2     10000
14     9999
10     9999
5      9928
24     9508
6      8888
18     6501
Name: cluster, dtype: int64

In [523]:
new=final_df.copy()
new['last_sale_price']=np.log(new['last_sale_price'])

### Solution 4

In [524]:
alg_list=[gbrt,regressor,light_gbm]
names=['GBRT','XGB','LGBM']
for x in range(len(alg_list)):
  panel_temp=panel(new,alg_list[x],'cluster',rmse,'last_sale_price',test)
  if x==0:
    stack=panel_temp[final_df.shape[0]-test.shape[0]:]
  else:
    out=panel_temp[final_df.shape[0]-test.shape[0]:]
    stack=pd.concat([stack,out],axis='columns')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
[07:40:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[07:40:12] INFO: /workspace/src/

In [525]:
from numpy.random import SFC64
stack.columns=names
S4=np.exp(stack)
S4.head()

,GBRT,XGB,LGBM
309910,0.963151,1.016119,1.167908
309911,0.681500,1.006485,0.754318
309912,2.973078,3.576132,2.991485
309913,0.963151,1.040840,1.258586
309914,0.083184,0.852089,1.109089


In [526]:
S4['last_sale_price']=(S4['GBRT']+S4["XGB"]+S4['LGBM'])/3

In [527]:
S4.head()

,GBRT,XGB,LGBM,last_sale_price
309910,0.963151,1.016119,1.167908,1.049059
309911,0.681500,1.006485,0.754318,0.814101
309912,2.973078,3.576132,2.991485,3.180232
309913,0.963151,1.040840,1.258586,1.087526
309914,0.083184,0.852089,1.109089,0.681454


### Solution 5

In [528]:
pre_tuned_etr={0: ExtraTreesRegressor(max_depth=12, min_samples_leaf=20, min_samples_split=25,
                    n_jobs=4),
               1:ExtraTreesRegressor(max_depth=5, min_samples_leaf=80, min_samples_split=25,
                    n_estimators=55, n_jobs=4),
               2:ExtraTreesRegressor(max_depth=4, min_samples_leaf=40, min_samples_split=25,
                    n_estimators=55, n_jobs=4),
               6:ExtraTreesRegressor(max_depth=4, min_samples_leaf=40, min_samples_split=25,
                    n_estimators=40, n_jobs=4),
               7:ExtraTreesRegressor(max_depth=8, min_samples_leaf=80, min_samples_split=25,
                    n_jobs=4),
               10:ExtraTreesRegressor(max_depth=4, min_samples_leaf=20, min_samples_split=25,
                    n_estimators=40, n_jobs=4),
               16: ExtraTreesRegressor(max_depth=2, min_samples_leaf=10, min_samples_split=25,
                    n_jobs=4),
               24: ExtraTreesRegressor(max_depth=12, min_samples_leaf=20, min_samples_split=25,
                    n_jobs=4)}
pre_tuned_gbm={0:GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_state=42),
               1:GradientBoostingRegressor(max_depth=6, min_samples_split=125, n_estimators=60,
                          random_state=42),
               2:GradientBoostingRegressor(max_depth=4, min_samples_split=250, n_estimators=50,
                          random_state=42),
               6: GradientBoostingRegressor(max_depth=5, min_samples_split=150, n_estimators=50,
                          random_state=42),
               7:GradientBoostingRegressor(max_depth=5, min_samples_split=125, n_estimators=55,
                          random_state=42),
               10: GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_state=42),
               16: GradientBoostingRegressor(max_depth=4, min_samples_split=250, n_estimators=50,
                          random_state=42),
               24: GradientBoostingRegressor(max_depth=6, min_samples_split=100, n_estimators=80,
                          random_state=42)}
pre_tuned_xgb={0:XGBRegressor(base_score=0.75, booster='gblinear', gpu_id=1,
             interaction_constraints='', min_child_weight=4,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               1:XGBRegressor(base_score=0.25, gpu_id=1, interaction_constraints='',
             min_child_weight=3, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3),
               2:XGBRegressor(base_score=0.25, gpu_id=1, interaction_constraints='',
             min_child_weight=3, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3),
               6:XGBRegressor(base_score=0.75, booster='gblinear', gpu_id=1,
             interaction_constraints='', min_child_weight=4,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               7:XGBRegressor(base_score=0.75, booster='gblinear', gpu_id=1,
             interaction_constraints='', min_child_weight=4,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               10: XGBRegressor(base_score=1, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2, max_depth=10,
             min_child_weight=3, monotone_constraints='(1,-1)',
             n_estimators=1500, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               16:XGBRegressor(base_score=0.25, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2, max_depth=10,
             min_child_weight=2, monotone_constraints='(1,-1)',
             n_estimators=1500, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3) ,
               24: XGBRegressor(booster='gblinear', gpu_id=1, interaction_constraints='',
             learning_rate=0.05, min_child_weight=4,
             monotone_constraints='(1,-1)', n_estimators=1100, n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3)}


pre_tuned_lgbm={0:LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.5, feature_fraction_seed=7, lambda_l1=10,
              learning_rate=0.2, max_bin=200, max_depth=12, min_data_in_leaf=50,
              n_estimators=60, num_leaves=10, objective='regression',
              verbose=-1),

               1:LGBMRegressor(bagging_fraction=0.7, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.2, feature_fraction_seed=7, lambda_l1=2,
              learning_rate=0.05, max_bin=200, max_depth=8, min_data_in_leaf=40,
              n_estimators=80, num_leaves=40, objective='regression',
              verbose=-1),

               2:LGBMRegressor(bagging_fraction=0.9, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.5, feature_fraction_seed=7, lambda_l1=10,
              max_bin=200, max_depth=10, min_data_in_leaf=20, num_leaves=3,
              objective='regression', verbose=-1),

               6:LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.2, feature_fraction_seed=7, lambda_l1=0,
              learning_rate=0.02, max_bin=200, max_depth=10,
              min_data_in_leaf=10, n_estimators=40, num_leaves=40,
              objective='regression', verbose=-1),

               7:LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.5, feature_fraction_seed=7, lambda_l1=0,
              learning_rate=0.03, max_bin=200, max_depth=10,
              min_data_in_leaf=40, n_estimators=50, num_leaves=5,
              objective='regression', verbose=-1),

               10:LGBMRegressor(bagging_fraction=0.9, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.5, feature_fraction_seed=7, lambda_l1=2,
              learning_rate=0.05, max_bin=200, max_depth=2, min_data_in_leaf=10,
              n_estimators=30, num_leaves=40, objective='regression',
              verbose=-1),

               16: LGBMRegressor(bagging_fraction=0.9, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.75, feature_fraction_seed=7, lambda_l1=2,
              learning_rate=0.01, max_bin=200, max_depth=2, min_data_in_leaf=30,
              n_estimators=80, num_leaves=10, objective='regression',
              verbose=-1),

               24: LGBMRegressor(bagging_fraction=0.7, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.75, feature_fraction_seed=7, lambda_l1=0,
              learning_rate=0.02, max_bin=200, max_depth=12,
              min_data_in_leaf=10, n_estimators=200, num_leaves=30,
              objective='regression', verbose=-1)}
                    

In [529]:
tuned_list=[pre_tuned_etr,pre_tuned_gbm,pre_tuned_xgb,pre_tuned_lgbm]
hyp=[random_cv_etr,random_cv_gbm,random_cv_xgb,random_cv_lgbm]
for x in range(len(tuned_list)):
  panel_temp=panel_pre_tune(new,tuned_list[x],'cluster',rmse,'last_sale_price',test)
  if x==0:
    stack=panel_temp[final_df.shape[0]-test.shape[0]:]
  else:
    out=panel_temp[final_df.shape[0]-test.shape[0]:]
    stack=pd.concat([stack,out],axis='columns')

0
(10025, 39) (9999, 39)
ExtraTreesRegressor(max_depth=12, min_samples_leaf=20, min_samples_split=25,
                    n_jobs=4)
1
(23788, 39) (3137, 39)
ExtraTreesRegressor(max_depth=5, min_samples_leaf=80, min_samples_split=25,
                    n_estimators=55, n_jobs=4)
2
(4010, 39) (9999, 39)
ExtraTreesRegressor(max_depth=4, min_samples_leaf=40, min_samples_split=25,
                    n_estimators=55, n_jobs=4)
3
(9928, 39) (0, 39)
4
(6501, 39) (0, 39)
5
(15555, 39) (0, 39)
6
(27077, 39) (7827, 39)
ExtraTreesRegressor(max_depth=4, min_samples_leaf=40, min_samples_split=25,
                    n_estimators=40, n_jobs=4)
7
(39320, 39) (1457, 39)
ExtraTreesRegressor(max_depth=8, min_samples_leaf=80, min_samples_split=25,
                    n_jobs=4)
8
(19602, 39) (0, 39)
9
(11110, 39) (0, 39)
10
(24158, 39) (30, 39)
ExtraTreesRegressor(max_depth=4, min_samples_leaf=20, min_samples_split=25,
                    n_estimators=40, n_jobs=4)
11
(9999, 39) (0, 39)
12
(8888, 39) (0,

In [530]:
names=['ETR','GBM','XGB','LGBM']
stack.columns=names
S5=np.exp(stack)
S5.head()

,ETR,GBM,XGB,LGBM
309910,1.128742,0.897923,0.821552,0.983885
309911,1.412517,0.944575,0.991122,1.345546
309912,2.229983,2.837737,3.110667,2.454683
309913,1.128742,0.897923,0.821552,0.955636
309914,1.139269,0.363728,0.862179,0.965038


In [531]:
S5['last_sale_price']=(S5['ETR']+S5['GBM']+S5["XGB"]+S5['LGBM'])/4
S5.head()


,ETR,GBM,XGB,LGBM,last_sale_price
309910,1.128742,0.897923,0.821552,0.983885,0.958025
309911,1.412517,0.944575,0.991122,1.345546,1.173440
309912,2.229983,2.837737,3.110667,2.454683,2.658268
309913,1.128742,0.897923,0.821552,0.955636,0.950963
309914,1.139269,0.363728,0.862179,0.965038,0.832554


#### Exra Trees Solution Issue:
The original solution used was created from the same Extra Trees Algorithm, however Extra Trees Provide different solutions each time. error of solution calculated shows that the solutions are extrremely close

In [539]:
result32=pd.read_csv('/content/drive/MyDrive/NFT Price Prediction/dataset/result32_panel_tune.csv')
result32.head()

,global_index,last_sale_price
0,0.0,0.960239
1,1.0,1.159658
2,2.0,2.664536
3,3.0,0.953177
4,4.0,0.833982


In [541]:
rmse(np.array(result32['last_sale_price']), np.array(S5['last_sale_price']))

0.9975439849159554

# Final

In [459]:
df_result4=pd.DataFrame(S4['last_sale_price'].to_list(),columns=["last_sale_price"]) 
df_result5=pd.DataFrame(S5['last_sale_price'].to_list(),columns=["last_sale_price"]) 

In [460]:
df_result4.head(5)

,last_sale_price
0,1.049059
1,0.814101
2,3.180232
3,1.087526
4,0.681454


In [461]:
df_result5.head(5)

,last_sale_price
0,0.963775
1,1.186527
2,2.678030
3,0.956713
4,0.838865


In [462]:
s=pd.read_csv('/content/drive/MyDrive/NFT Price Prediction/dataset/submission_format.csv')

In [463]:
s.head()

,global_index,last_sale_price
0,0,0.234588
1,1,0.243322
2,2,0.231398
3,3,0.150778
4,4,2.015249


In [464]:
result4=pd.concat([s['global_index'],df_result4],axis='columns')
result5=pd.concat([s['global_index'],df_result5],axis='columns')

In [465]:
result4.head()

,global_index,last_sale_price
0,0.0,1.049059
1,1.0,0.814101
2,2.0,3.180232
3,3.0,1.087526
4,4.0,0.681454


In [466]:
result5.head()

,global_index,last_sale_price
0,0.0,0.963775
1,1.0,1.186527
2,2.0,2.678030
3,3.0,0.956713
4,4.0,0.838865


In [542]:
result4.to_csv('solution4.csv',index=False)
result5.to_csv('solution5.csv',index=False)